In [48]:
# import dependancies
import pandas as pd
import random
from datetime import datetime, timedelta
from yahoo_fin.stock_info import *
import yfinance as yf
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from splinter import Browser
from pprint import pprint
import finnhub
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# change to your api keys 

#alphavantage_api_key = 'AJP4EKR4LE52NNK4'
alphavantage_api_key = 'CORYDT1RMNVL3DVT'
polygon_api_key = 'J9UUfkyTylf3D_s76b5pcMCnLSaKTNpD'
finnhub_client = finnhub.Client(api_key="cld35g9r01qn959jif1gcld35g9r01qn959jif20")


### Screening US Mega & Large Cap Stocks

In [49]:
file_path = 'nasdaq_screener_1700335987448.csv'

# Read the CSV file into a Pandas DataFrame
stock_list_df = pd.read_csv(file_path)

# Display the DataFrame
stock_list_df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$113.15,-1.04,-0.911%,3.310625e+10,United States,1999.0,2700770,Industrials,Electrical Products
1,AAPL,Apple Inc. Common Stock,$189.69,-0.02,-0.011%,2.950202e+12,United States,1980.0,50922657,Technology,Computer Manufacturing
2,ABBV,AbbVie Inc. Common Stock,$138.30,0.02,0.014%,2.441738e+11,United States,2012.0,4332297,Health Care,Biotechnology: Pharmaceutical Preparations
3,ABNB,Airbnb Inc. Class A Common Stock,$127.15,0.87,0.689%,8.244890e+10,United States,2020.0,3795704,Consumer Discretionary,Hotels/Resorts
4,ABT,Abbott Laboratories Common Stock,$99.55,-0.71,-0.708%,1.728246e+11,United States,NaN,6069070,Health Care,Biotechnology: Pharmaceutical Preparations


In [50]:
# create an array of tickers

ticker_list = stock_list_df['Symbol'].values

### Generating functions to retrieve stock information 

In [51]:
# define function to generate random date, and ensure that the date is a weekday, maybe start in 2014 or 2015

def generate_random_date(start_date, end_date):
    check_weekday = False
    while not check_weekday:
        # Convert the date strings to datetime objects
        start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date_dt = datetime.datetime.now()  # Using today's date as the end date

        # Calculate the range of days between the start and end date
        delta_days = (end_date_dt - start_date_dt).days

        # Generate a random number of days within the range
        random_days = random.randint(0, delta_days)

        # Calculate the random date
        random_date = start_date_dt + timedelta(days=random_days)

        # Check if the generated date is a weekday
        if random_date.weekday() < 5:
            check_weekday = True
            return random_date
        else:
            check_weekday = False



In [52]:
# define a function to choose a random stock from the list

def choose_random_stock(ticker_list):    
    
    # Randomly choose a stock from the list
    random_stock = random.choice(ticker_list)
    
    return random_stock.lower()
    

In [53]:
# define a function to check if data exists for stock on the date 

def check_stock_date(ticker, check_date):
    check_financials = False
    # Check if the input date is a Friday
    if check_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        adj_end_date = check_date + timedelta(days=3)
    else:
        # Set end date to the next day
        adj_end_date = check_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    
    
    try:
        get_data(ticker, end_date=adj_end_date)
        get_stock_financials(ticker, check_date)
        

        check_financials = True
    except:
        check_financials = False
    
    if not check_financials:
        return False
    else:
        return True
    

In [54]:
# define a function to return stock industry

def return_stock_industry(symbol):
    symbol = symbol.upper()
    # Check if the symbol is in the DataFrame
    if symbol in stock_list_df['Symbol'].values:
        # Retrieve the sector for the given symbol
        sector = stock_list_df.loc[stock_list_df['Symbol'] == symbol, 'Sector'].iloc[0]
        return sector
    else:
        return f"Symbol '{symbol}' not found in the stock list."

In [55]:

# define a function to get the adjusted closing price of a stock on a particular day

def get_stock_price(ticker, stock_date):
    # Check if the input date is a Friday
    if stock_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        price_adj_end_date = stock_date + timedelta(days=3)
    else:
        # Set end date to the next day
        price_adj_end_date = stock_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=price_adj_end_date)
    # extracting the price 
    adj_close_price = stock_data['adjclose'].values[-1]
    
    return adj_close_price

In [56]:
# define a function to get stock trading volume

def get_stock_volume(ticker, vol_date):
    # Check if the input date is a Friday
    if vol_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        vol_adj_end_date = vol_date + timedelta(days=3)
    else:
        # Set end date to the next day
        vol_adj_end_date = vol_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=vol_adj_end_date)
    
    #extracting the volume
    trading_vol = stock_data['volume'].values[-1]
    
    return trading_vol

In [57]:
# function to get 4 previous weekdays from given_date

def get_previous_weekdays(weekday_date):
    previous_weekdays = []
    previous_date = weekday_date - timedelta(days=1)
    while len(previous_weekdays) < 4:
        if previous_date.weekday() < 5:
            previous_weekdays.append(previous_date)
        previous_date -= timedelta(days=1)

    return previous_weekdays


In [58]:
# function to get company's financial ratios by scraping stockanalysis.com

def get_stock_financials(ticker, fin_date):
    
    financial_ratios = {}

    # setting brwoser
    browser = Browser('chrome')
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    table = soup.find("table", class_="w-full")

    # Check if the table is found
    if table:
        # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        
    else:
        print("Table not found on the page.")
    
    # Extract date columns excluding the first column
    date_columns = df.columns[2:-1]
        
    # Convert date columns to datetime objects
    date_columns = pd.to_datetime(date_columns)
        
    # Find the closest date column to the given date that is earlier
    earlier_dates = [date for date in date_columns if date < fin_date]
    closest_date_column = max(earlier_dates)

    # Access the closest column
    closest_column = closest_date_column.strftime('%Y-%m-%d')

    #extracting financial ratios
    
    financial_ratios['pe_ratio']=float(df.loc[df['Quarter Ended'] == 'PE Ratio', closest_column].values[0])
    financial_ratios['debt_to_equity']=float(df.loc[df['Quarter Ended'] == 'Debt / Equity Ratio', closest_column].values[0])
    financial_ratios['quick_ratio']=float(df.loc[df['Quarter Ended'] == 'Quick Ratio', closest_column].values[0])
    total_shareholder_return_str = df.loc[df['Quarter Ended'] == 'Total Shareholder Return', closest_column].values[0]
    financial_ratios['total_shareholder_return'] = float(total_shareholder_return_str.strip('%'))
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    # Find the table by class name
    table = soup.find("table", class_="w-full")

    # Check if the table is found
    if table:
        # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        
    else:
        print("Table not found on the page.")
    
    profit_margin_str= df.loc[df['Quarter Ended'] == 'Profit Margin', closest_column].values[0]
    financial_ratios['profit_margin'] = float(profit_margin_str.strip('%'))
    cash_margin_str= df.loc[df['Quarter Ended'] == 'Free Cash Flow Margin', closest_column].values[0]
    financial_ratios['free_cash_margin'] = float(cash_margin_str.strip('%'))
    

    return financial_ratios
    
    

In [59]:
# function to calculate stock's volatility

def calculate_volatility(ticker, sd_date):
    df=get_data(ticker, end_date=sd_date)
    # Calculate daily returns
    df['daily_return'] = df['adjclose'].pct_change()

    # Calculate volatility as the standard deviation of daily returns
    volatility = df['daily_return'].std()

    return volatility

In [13]:
#------------------------NEED TO FIND A NEW SOURCE---------------------------------

def get_news_sentiment(ticker, news_date):
    
    # Extract year, month, and day
    news_end_date=news_date
    end_year = news_end_date.year
    end_month = str(news_end_date.month).zfill(2)
    end_day = str(news_end_date.day).zfill(2)
    
    news_start_date = news_end_date - timedelta(weeks=4)
    start_year = news_start_date.year
    start_month = str(news_start_date.month).zfill(2)
    start_day = str(news_start_date.day).zfill(2)
   

In [60]:
# extracting daily federal interest rate information from alphavantage

# using alphavantage API
url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=daily&apikey={alphavantage_api_key}'
r = requests.get(url)
data = r.json()

daily_interest_rates = data

# converting json to dataframe
daily_interest_rate_df = pd.DataFrame(daily_interest_rates['data'])

# Export DataFrame to CSV so that we can refer to it without making an api call
daily_interest_rate_df.to_csv('daily_interest_rates.csv', index=False)


In [61]:
# extracting monthly CPI information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_cpi = data

# converting json to dataframe
monthly_cpi_df = pd.DataFrame(monthly_cpi['data'])
monthly_cpi_df['date'] = pd.to_datetime(monthly_cpi_df['date'])


# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_cpi_df.to_csv('monthly_cpi.csv', index=False)

In [62]:
# extracting monthly unemployment rate information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_unemployment_rate = data

# converting json to dataframe
monthly_unemployment_rate_df = pd.DataFrame(monthly_unemployment_rate['data'])
monthly_unemployment_rate_df['date'] = pd.to_datetime(monthly_unemployment_rate_df['date'])

# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_unemployment_rate_df.to_csv('monthly_unemployment_rate.csv', index=False)

In [63]:
# define a function to extract economic indicators for the given date

def get_economic_indicators(econ_date):
    
    economic_indicators_dict={}
    
    # convert datetime object to matching string 
    formatted_date = econ_date.strftime('%Y-%m-%d')
    
    # retrieve CPI value
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['cpi'] = float(monthly_cpi_df.loc[monthly_cpi_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])    
    
    # retrieve federal interest rate value
    economic_indicators_dict['interest_rate'] = float(daily_interest_rate_df.loc[daily_interest_rate_df['date'] == formatted_date, 'value'].iloc[0])

    # retrieve unemployement rate
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['unemployment_rate'] = float(monthly_unemployment_rate_df.loc[monthly_unemployment_rate_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])
    
    return economic_indicators_dict

In [64]:
# function to retrieve technical indicators

def get_technical_indicators(ticker, tech_date):
    
    technical_indicators_dict={}
    
    # changing date into UNIX format
    end_timestamp = int(tech_date.timestamp())
    
    # choosing a timestamp of one week earlier
    start_tech_date = tech_date - timedelta(weeks=1)
    
    # changing the date to UNIX format
    start_timestamp = int(start_tech_date.timestamp())
    
    # retrieving SMA 
    technical_indicators_dict['sma']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='sma', indicator_fields={"timeperiod": 5})['sma'][-1]
    
    #retrieving RSI
    technical_indicators_dict['rsi']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='rsi', indicator_fields={"timeperiod": 5})['rsi'][-1]

    # retrieving EMA 
    technical_indicators_dict['ema']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='ema', indicator_fields={"timeperiod": 5})['ema'][-1]
    
    return technical_indicators_dict

In [65]:
# function to create label value for data

def create_label(ticker, y_date):
    print(y_date)
    initial_price = get_stock_price(ticker, y_date)
    print(initial_price)
    
    # find the next weekday
    
    # check if date landed on friday and calculate the next weekday date
    if y_date.weekday() == 4:
        next_weekday_date = y_date + timedelta(days=3)
    else:
        next_weekday_date = y_date + timedelta(days=1)

    print(next_weekday_date)
    
    next_price = get_stock_price(ticker, next_weekday_date)
    
    print(next_price)
    
    if next_price >= initial_price:
        return 1 # 1 is class increase, in order to simplify the classification we put stayed the same in same category as increase
    else:
        return 0 # 0 is class decrease

### Running a loop to generate multiple rows of data for training and testing

In [21]:
page=10

In [69]:
# list to store dictionaries
list_dict = []
page += 1
# runs loop until list_dic [change no. 4 as desired]
while len(list_dict) < 10:

    # data_dict
    data_dict = {}
    
    # checking to see if generated ticker-date combo has available data
    checker = False
    
    # runs this loop until checker is true
    while not checker:

        ticker = choose_random_stock(ticker_list)
        input_date = generate_random_date('2015-01-01', '2023-10-31')

        checker = check_stock_date(ticker, input_date)

    # using function to return stock industry
    data_dict['industry']=return_stock_industry(ticker)

    # retrieve weekly returns
    date = input_date
    for i in range(1, 5):
        data_dict[f'wr{i}']=(get_stock_price(ticker, date) - get_stock_price(ticker, date - timedelta(weeks = 1)))/get_stock_price(ticker, date - timedelta(weeks = 1))
        date = date - timedelta(weeks=1)

    # retrieve last 5 working day volumes
    data_dict['vol1'] = get_stock_volume(ticker, date)
    for i, weekday in enumerate(get_previous_weekdays(input_date)):
        data_dict[f'vol{i+1}'] = get_stock_volume(ticker, weekday)

    # retrieve stock financials    
    for key, value in get_stock_financials(ticker, input_date).items():
        data_dict[key] = value
    
    # retrieve stock volatility
    data_dict['volatility'] = calculate_volatility(ticker, input_date)

    # retrieve economic indicators
    for key, value in get_economic_indicators(input_date).items():
        data_dict[key] = value

    # retrieve stocktechnical indicators    
    for key, value in get_technical_indicators(ticker, input_date).items():
        data_dict[key] = value

    # generate a label for row    
    data_dict['label'] = create_label(ticker, input_date)
        
    # append dictionary to list
    list_dict.append(data_dict)


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

2017-06-02 00:00:00
62.24113464355469
2017-06-05 00:00:00
62.15579605102539


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

2021-10-05 00:00:00
97.16999816894531
2021-10-06 00:00:00
97.11000061035156


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

...

Table not found on the page.


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

2022-06-16 00:00:00
79.36767578125
2022-06-17 00:00:00
79.86762237548828


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\1179120743.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

2020-04-23 00:00:00
127.66999816894531
2020-04-24 00:00:00
128.86000061035156


In [37]:
data_dict

{'industry': 'Health Care',
 'wr1': -0.031820758027566697,
 'wr2': -0.005275493827999014,
 'wr3': -0.002241916472637536,
 'wr4': -0.011276076414879422,
 'vol1': 4327000,
 'vol2': 3518000,
 'vol3': 3871000}

In [70]:
list_dict

[{'industry': 'Utilities',
  'wr1': 0.03335499121218741,
  'wr2': 0.025895441166562158,
  'wr3': -0.0001320732861866999,
  'wr4': -0.010296082453015798,
  'vol1': 1684300,
  'vol2': 2183700,
  'vol3': 1040600,
  'vol4': 1065800,
  'pe_ratio': -18.64,
  'debt_to_equity': 2.09,
  'quick_ratio': 0.4,
  'total_shareholder_return': 4.02,
  'profit_margin': 3.19,
  'free_cash_margin': -11.69,
  'volatility': 0.015272056670616473,
  'cpi': 244.955,
  'interest_rate': 0.91,
  'unemployment_rate': 4.3,
  'sma': 79.72333333333334,
  'rsi': 100,
  'ema': 79.66666666666666,
  'label': 0},
 {'industry': 'Consumer Discretionary',
  'wr1': 0.12621696272278488,
  'wr2': 0.0002318182537814634,
  'wr3': -0.024539162744052406,
  'wr4': -0.04109738522402556,
  'vol1': 6853200,
  'vol2': 7063700,
  'vol3': 15163900,
  'vol4': 26441400,
  'pe_ratio': 15.54,
  'debt_to_equity': 1.38,
  'quick_ratio': 0.21,
  'total_shareholder_return': 2.23,
  'profit_margin': 4.45,
  'free_cash_margin': -0.77,
  'volatility

### Export data into csv

In [71]:
# remember to export data as csv
data_df = pd.DataFrame(list_dict)
data_df.to_csv(f'data{page}.csv', index=False)


In [72]:
data_df

,industry,wr1,wr2,wr3,wr4,vol1,vol2,vol3,vol4,pe_ratio,...,profit_margin,free_cash_margin,volatility,cpi,interest_rate,unemployment_rate,sma,rsi,ema,label
0,Utilities,0.033355,0.025895,-0.000132,-0.010296,1684300,2183700,1040600,1065800,-18.64,...,3.19,-11.69,0.015272,244.955,0.91,4.3,79.723333,100.000000,79.666667,0
1,Consumer Discretionary,0.126217,0.000232,-0.024539,-0.041097,6853200,7063700,15163900,26441400,15.54,...,4.45,-0.77,0.026291,276.589,0.08,4.5,97.840000,69.509465,97.196250,0
2,Consumer Discretionary,-0.019770,0.006916,0.001310,-0.000935,403800,330100,194200,170800,24.73,...,2.75,10.61,0.022560,233.707,0.12,5.7,108.230000,25.000000,107.196667,1
3,Utilities,0.012761,0.004863,0.011630,0.024282,1841500,1582200,1570300,1223700,13.24,...,13.48,5.49,0.016841,233.707,0.11,5.7,68.346667,74.635726,68.514167,0
4,Consumer Discretionary,-0.013342,0.039220,-0.025480,-0.008896,226300,273100,188800,415500,37.89,...,5.31,18.98,0.028124,241.432,0.66,4.7,103.526667,37.661098,103.540833,0
5,Miscellaneous,-0.000074,-0.005090,0.052489,0.013057,323900,476100,501000,268200,23.80,...,2.04,-1.87,0.021680,254.202,2.40,3.8,135.340000,43.347242,135.132917,0
6,Energy,-0.084881,0.016172,0.174051,-0.036585,5026200,6523700,7429800,10657800,33.59,...,-1.56,-58.88,0.031282,257.208,1.55,3.6,3.503333,14.814815,3.503750,0
7,Consumer Discretionary,0.014301,0.006608,-0.015489,-0.080276,55528600,40741800,42178600,38773700,65.32,...,6.90,6.93,0.035843,307.671,5.33,3.9,131.316667,44.702467,130.527083,1
8,Consumer Staples,-0.032353,-0.030678,-0.014517,0.067047,1979000,2132500,1778200,1623100,8.14,...,6.32,-4.93,0.023662,296.311,1.58,3.6,83.526667,13.973458,83.324167,1
9,Health Care,0.025791,-0.031666,0.226665,-0.052194,634300,1077200,1180100,1486300,96.49,...,2.98,0.80,0.017139,256.389,0.04,14.7,125.073333,64.257703,126.453333,1


In [21]:
page = 1